In [ ]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve, auc, roc_auc_score
from sklearn.model_selection import KFold
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
import torch
import gc
import re
from tqdm import tqdm

tqdm.pandas()

In [ ]:
text_and_bert = pd.read_parquet('/kaggle/extracted_data/text_and_bert.parquet', engine='pyarrow')
text_and_bert['description'] = text_and_bert['description'].fillna('no desc')

In [ ]:
attrs = pd.read_parquet('/kaggle/input/extracted_data/attributes.parquet', columns=['categories', 'characteristic_attributes_mapping'], engine='pyarrow')
attrs['category_level_2'] = attrs['categories'].progress_apply(lambda x: eval(x)['2'])

In [ ]:
data = pd.concat([text_and_bert, attrs], axis=1)

In [ ]:
del text_and_bert, attrs
gc.collect()

In [ ]:
def remove_html_tags_and_emoji(text):
    if text is None:
        return None
    clean = re.compile('<.*?>')
    text = re.sub(clean, '', text)
    text = text.replace('\n', ' ')
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U0001F1E0-\U0001F1FF"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

data['description'] = data['description'].progress_apply(remove_html_tags_and_emoji)
data['name'] = data['name'].progress_apply(remove_html_tags_and_emoji)
data['category_level_2'] = data['category_level_2'].progress_apply(lambda x: x.lower())

In [ ]:
train_pairs = pd.read_parquet('/kaggle/input/extracted_data/train.parquet', engine='pyarrow')

In [ ]:
train_pairs.rename(
    columns={
        'variantid1': 'variantid_1',
        'variantid2': 'variantid_2'
    }, inplace=True
)

train_df = train_pairs.merge(
    data.add_suffix('_1'), 
    on='variantid_1'
).merge(
    data.add_suffix('_2'), 
    on='variantid_2'
)

In [ ]:
train_df['category_level_2'] = train_df['category_level_2_1']
train_df.drop(columns=['category_level_2_1', 'category_level_2_2'], axis=1, inplace=True)

In [ ]:
def get_dist_and_sim(dict1, dict2):
    dist, sim = [], []
    try:
        dict1, dict2 = eval(dict1), eval(dict2)
        dict_keys = set(dict1.keys()) & set(dict2.keys())
    except:
        return dist, sim
    
    for key in dict_keys:
        val1 = dict1.get(key)
        val2 = dict2.get(key)
        if val1 != val2:
            dist.append(key)
        if val1 == val2:
            sim.append(key)
    return dist, sim

dataset = []
for i in tqdm(range(len(train_df))):
    row = train_df.iloc[i]
    target = row.target
    category = row.category_level_2
    name1 = row.name_1
    name2 = row.name_2
    desc1 = row.description_1
    desc2 = row.description_2
    res_dist, res_similar = get_dist_and_sim(
        row.characteristic_attributes_mapping_1,
        row.characteristic_attributes_mapping_2
    )
    dataset.append(
        (category, 
         name1, 
         name2,
         desc1, 
         desc2, 
         ', '.join(res_dist), 
         ', '.join(res_similar), 
         target)
    )

In [ ]:
model_name = 'cointegrated/rubert-tiny2'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).cuda()

batch_size = 32
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
epochs = 3
total_steps = (1 + len(dataset) // batch_size) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0, 
                                            num_training_steps=total_steps)

train_losses = []
for ep in range(epochs):
    np.random.shuffle(dataset)
    optimizer.zero_grad()
    losses = []
    pbar = tqdm(range(len(dataset)), desc=f'Epoch {ep} Loss 0.000', total=len(dataset))
    for t in pbar:
        category, name1, name2, desc1, desc2, dist, sim, target = dataset[t]
        name = name1 + '[SEP]' + name2
        desc_start = desc1[:300] + '[SEP]' + desc2[:300]
        desc_end = desc1[-300:] + '[SEP]' + desc2[-300:] 
        s = category + '[SEP]' + 'Названия: ' + name + '[SEP]' + 'Описания: ' + desc_start + '[SEP]' + desc_end
        tks = tokenizer.encode_plus(
            s[:1800], 
            max_length=1024, 
            pad_to_max_length=False, 
            return_attention_mask=True, 
            return_tensors='pt', 
            truncation=True
        )
        out = model(
            tks['input_ids'].cuda(), 
            attention_mask=tks['attention_mask'].cuda(),
            token_type_ids=tks['token_type_ids'].cuda(),
            labels=torch.tensor([[1.0 - target, target]]).float().cuda()
        )
        
        losses.append(out.loss)
        
        if (t + 1) % batch_size == 0:
            loss = sum(losses) / batch_size
            loss.backward()
            losses = []
            train_losses.append(loss.item())
            optimizer.step() 
            optimizer.zero_grad()
            scheduler.step()
            pbar.set_description(f'Epoch {ep} Loss {loss.item():.3f}')
    
    if len(losses) > 0:
        loss = sum(losses) / batch_size
        loss.backward()
        losses = []
        train_losses.append(loss.item())
        optimizer.step() 
        optimizer.zero_grad()
        scheduler.step()
        pbar.set_description(f'Epoch {ep} Loss {loss.item():.3f}')

torch.save(model.state_dict(), f'name_desc_bert_full.pth')